# PostgreSQL 16.6+

## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意（`ORDER BY` なし）
* `NOT IN` 回避（本問では未使用）
* 判定は **ID（= `name` 主キー）基準**、表示は仕様どおり

## 1) 問題

* `面積が 3,000,000 以上 または 人口が 25,000,000 以上の国（big）を抽出し、name, population, area を返す`
* 入力: `World(name PK, continent, area INT, population INT, gdp BIGINT)`
* 出力: `name, population, area`（順不同）

## 2) 最適解（単一クエリ）

> 本件はウィンドウ不要。シンプルな選択で十分（PostgreSQL では BitmapOr により `OR` でも複数インデックスが活用されやすい）。

```sql
SELECT
  name,
  population,
  area
FROM World
WHERE area >= 3000000
   OR population >= 25000000;

Runtime 295 ms
Beats 75.25%

```

### 代替（`UNION` で個別インデックスを強制活用したい場合）

```sql
SELECT name, population, area
FROM World
WHERE area >= 3000000

UNION   -- 両条件を満たす国の重複を排除

SELECT name, population, area
FROM World
WHERE population >= 25000000;

Runtime 284 ms
Beats 94.53%

```

> `UNION ALL` ではなく `UNION` を使用して重複排除。

### 代替（LATERAL で上位 k を作る）

> 本問題は **上位 k 取得ではないため不要**。LATERAL の出番はありません。

## 3) 要点解説

* **方針**: 単純フィルタのみで達成。結果順は任意なので `ORDER BY` を付けずに I/O を抑制。
* **PostgreSQL 特性**: プランナは `OR` 条件で **BitmapOr** を選択し、`area`・`population` の双方のインデックスを併用することが多い。データ分布次第で `UNION` 版よりシンプルな `OR` 版が速い場合もある。
* **NULL**: `area` / `population` が `NULL` の行は比較が `UNKNOWN` となり除外（仕様上問題なし）。
* **出力列**: 仕様どおり `name, population, area` の順で投影。

## 4) 計算量（概算）

* `WHERE` 版: フルスキャン時 **O(N)**、インデックス利用時は **O(log N + K)** 近似。
* `UNION` 版: それぞれの枝で **O(log N + K₁)** / **O(log N + K₂)**、`UNION` の重複排除で **O(K log K)** 程度。

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[World] --> B[WHERE area>=3000000 <br>OR population>=25000000]
  B --> C[出力 name population area]
```
